In [0]:
# %pip install (notebook 환경에 필요한 모듈)
# openai
# nest_asyncio
# aiohttp
# azure-communication-email
# markdown

%pip install openai nest_asyncio aiohttp azure-communication-email markdown --no-cache-dir

In [0]:
dbutils.library.restartPython()

#### 전략 마케팅 가이드(원본)
마케팅 핵심 전략 8가지

In [0]:
STRATEGY_PLAYBOOK_GUIDE = {
  "Save": {
    "goal": "해지 직전/장기 미접속 유저 복귀",
    "tone_candidates": ["urgent"],
    "urgency_candidates": ["high"],
    "angles": [
      "놓치면 아쉬운 독점/화제작 한 편 제안",
      "취향 맞춤 한 편으로 복귀 장벽 낮추기",
      "바로 이어보기/짧게 시작 가능한 추천"
    ],
    "cta_candidates": ["지금 바로 이어보기", "내 취향으로 추천받기", "오늘의 추천 확인"],
    "do_not_say": ["지금 안 하면 손해", "마지막 경고", "무조건 할인", "해지하면 후회"]
  },
  "Start": {
    "goal": "가입 초기 정착 실패 복구",
    "tone_candidates": ["friendly", "helpful"],
    "urgency_candidates": ["medium", "low"],
    "angles": [
      "3분만에 취향 설정/추천 시작",
      "짧은 러닝타임 콘텐츠로 진입",
      "사용법/추천 흐름을 간단히 안내"
    ],
    "cta_candidates": ["3분만에 시작하기", "취향 선택하고 추천받기", "가볍게 한 편 보기"],
    "do_not_say": ["왜 안 보세요?", "사용법 모르죠?", "당장 결제하세요"]
  },
  "Fix": {
    "goal": "기술적 단절/불규칙 접속 해결",
    "tone_candidates": ["helpful"],
    "urgency_candidates": ["medium"],
    "angles": [
      "끊김 없는 시청 경험/기기 최적화 힌트",
      "최근 보던 작품 이어보기 제안",
      "간단한 해결 팁 + 바로 볼 콘텐츠"
    ],
    "cta_candidates": ["이어보기로 복귀", "설정 한 번에 해결", "바로 재생하기"],
    "do_not_say": ["고객님 문제입니다", "기기 탓", "불편하면 나가세요"]
  },
  "React": {
    "goal": "고위험 이용 감소 사용자 재참여",
    "tone_candidates": ["friendly", "urgent"],
    "urgency_candidates": ["high", "medium"],
    "angles": [
      "최근 취향 기반 ‘딱 맞는’ 추천",
      "짧게 시작 가능한 화제작",
      "다시 돌아오게 만드는 한 문장 훅"
    ],
    "cta_candidates": ["오늘 이거부터", "5분만 보고 결정", "지금 확인하기"],
    "do_not_say": ["왜 요즘 안 봐요", "당장 돌아와요"]
  },
  "Finish": {
    "goal": "몰입 후 급감 사용자 재개 유도",
    "tone_candidates": ["urgent", "premium"],
    "urgency_candidates": ["high", "medium"],
    "angles": [
      "이어보기/다음 회차로 몰입 재점화",
      "결말/후속 시즌 호기심 자극",
      "비슷한 작품으로 자연스러운 연장"
    ],
    "cta_candidates": ["바로 이어보기", "다음 회차 확인", "끝까지 몰아보기"],
    "do_not_say": ["안 보면 손해", "후회합니다"]
  },
  "Boost": {
    "goal": "고활동 유저 위험 증가 방지(충성도 유지)",
    "tone_candidates": ["premium"],
    "urgency_candidates": ["medium"],
    "angles": [
      "VIP 케어/취향 정교화",
      "신작/단독 공개/독점 라인업 강조",
      "고활동 유저용 ‘한 단계 위’ 추천"
    ],
    "cta_candidates": ["프리미엄 추천 받기", "신작 라인업 보기", "취향 더 정교하게"],
    "do_not_say": ["초보자 안내", "사용법 가이드 길게"]
  },
  "Stay": {
    "goal": "완만한 감소 추세 유지/회복",
    "tone_candidates": ["friendly", "calm"],
    "urgency_candidates": ["low", "medium"],
    "angles": [
      "부담 없는 추천(짧고 가벼운)",
      "주기적 시청 습관 회복",
      "편안한 톤으로 리마인드"
    ],
    "cta_candidates": ["가볍게 한 편", "오늘의 추천", "짧게 보기"],
    "do_not_say": ["긴급", "마지막 기회"]
  },
  "Calm": {
    "goal": "발송 억제(안정 상태)",
    "tone_candidates": ["calm"],
    "urgency_candidates": ["low"],
    "angles": ["(발송하지 않음)"],
    "cta_candidates": ["(none)"],
    "do_not_say": ["불필요한 메시지"]
  }
}

#### Playbook JSON 생성 프롬프트

In [0]:
def build_playbook_prompt(strategy_code: str,
                          probability_band: str,
                          priority_rank: int) -> str:
    guide = STRATEGY_PLAYBOOK_GUIDE.get(strategy_code, {})
    return f"""
        당신은 OTT 마케팅 전문가입니다.
        아래 입력을 바탕으로 최종 문구를 쓰기 전 '카피 설계도(Playbook)'를 **JSON만** 출력하세요.

        [입력]
        - strategy_code: {strategy_code}
        - probability_band: {probability_band}
        - priority_rank: {priority_rank}

        [전략 가이드]
        - 목표: {guide.get("goal")}
        - 추천 톤 후보: {guide.get("tone_candidates")}
        - 긴급도 후보: {guide.get("urgency_candidates")}
        - 앵글 예시: {guide.get("angles")}
        - CTA 예시: {guide.get("cta_candidates")}
        - 금지 표현: {guide.get("do_not_say")}

        [규칙]
        1) strategy_code는 절대 변경하지 마세요.
        2) 할인/쿠폰은 반드시 필요한 경우에만 사용하세요. 우선은 콘텐츠/경험 기반 대응을 고려하세요.
        3) 위협/압박/허위 혜택 금지.
        4) 반드시 한국어.
        5) JSON 외 텍스트 출력 금지.

        [출력 JSON 스키마]
        {{
        "hook": "10~18자 훅",
        "angle": "한 줄 요약",
        "content_direction": "추천 콘텐츠를 어떤 기준으로 활용할지",
        "cta": "짧은 행동 유도",
        "tone": "urgent|friendly|premium|calm|helpful",
        "urgency_level": "low|medium|high"
        }}
        """.strip()

In [0]:
def build_final_copy_prompt(user_situation: str,
                            strategy_code: str,
                            playbook_json: str,
                            content_list: str) -> str:
    guide = STRATEGY_PLAYBOOK_GUIDE.get(strategy_code, {})
    return f"""
        당신은 OTT 마케팅 전문가입니다. 아래 Playbook을 반드시 따르세요.

        [고객 상황]
        {user_situation}

        [전략 코드]
        {strategy_code}

        [금지 표현]
        {guide.get("do_not_say")}

        [Playbook(JSON)]
        {playbook_json}

        [실시간 추천 콘텐츠]
        {content_list}

        [항목 작성 규칙]
        push:
        1) 너무 길지 않게: 푸시 1~2줄(권장 60자 내외), 과장 금지.
        2) 추천 콘텐츠 제목/특징을 최소 1개 이상 자연스럽게 포함.
        3) 출력은 반드시 아래 형식만.
        email:
        1) 너무 길지 않게: 이메일 10~15줄(권장 300자 내외), 과장 금지.
        2) 추천 콘텐츠 리스트에 있는 영화의 제목과 특징을 문구에 자연스럽게 녹이세요.
        3) 반드시 한국어로 작성하세요.
        4) intro: 고객의 최근 시청 기록을 언급하며 흥미를 유발하는 3~4줄의 문구.
        5) rec_list: 추천 콘텐츠에 대해 각각 {{"title": "...", "description": "..."}}을 포함한 리스트 형식.
        6) description은 각 콘텐츠의 핵심 매력을 1문장으로 요약할 것.
        공통:
        1) 출력은 반드시 아래 형식만.

        [출력 형식]
        반드시 아래 JSON 형식으로만 답변하세요. 다른 텍스트는 포함하지 마세요.
        {{
            "push_title": "...",
            "push_body": "...",
            "email_title": "...",
            "email_intro": "...",
            "rec_list": [
                {{"title": "...", "description": "..."}},
                ...
            ]
        }}
        """.strip()

#### 엔드투엔드 실행 코드(노트북용, 비동기)
##### 필요 모듈 선언

In [0]:
import asyncio
import json
import time
from typing import Any, Dict, List, Optional, Tuple
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.window import Window

##### 0. 테이블 설정

In [0]:
CATALOG = "signalcraft_databricks"
SCHEMA  = "default"

TGT_TBL = f"{CATALOG}.{SCHEMA}.gold_campaign_targets"
SNP_TBL = f"{CATALOG}.{SCHEMA}.dlt_gold_user_behavior_snapshot"
EVT_TBL = f"{CATALOG}.{SCHEMA}.dlt_silver_watch_events_all"
CNT_TBL = f"{CATALOG}.{SCHEMA}.dlt_bronze_netflix_master"
OUT_TBL = f"{CATALOG}.{SCHEMA}.dlt_silver_campaign_messages"

##### 1. 전략 가이드/프롬프트
위에 선언한 STRATEGY_PLAYBOOK_GUIDE, build_playbook_prompt, build_final_copy_prompt 사용

##### 2. LLM 호출부(gpt-4o-mini)
Azure OpenAI로 구현(signalcraft-openai)

In [0]:
import nest_asyncio
from openai import AsyncAzureOpenAI
from azure.communication.email.aio import EmailClient

# 1. 설정 정보
SCOPE_NAME = "signalcraft-scope"

# OpenAI
AZURE_OPENAI_ENDPOINT = dbutils.secrets.get(scope=SCOPE_NAME, key="openai-endpoint")
AZURE_OPENAI_KEY = dbutils.secrets.get(scope=SCOPE_NAME, key="openai-api-key")
AZURE_OPENAI_DEPLOYMENT_TEXT = dbutils.secrets.get(scope=SCOPE_NAME, key="openai-deployment-text") # 배포한 모델명
AZURE_OPENAI_DEPLOYMENT_CHAT = dbutils.secrets.get(scope=SCOPE_NAME, key="openai-deployment-chat") # 배포한 모델명

# Notification
TEAMS_WEBHOOK_URL = dbutils.secrets.get(scope=SCOPE_NAME, key="teams-webhook-url")
ACS_EMAIL_ADDRESS = dbutils.secrets.get(scope=SCOPE_NAME, key="acs-email-address")
ACS_CONNECTION_STRING = dbutils.secrets.get(scope=SCOPE_NAME, key="acs-connection-string")

# 2. 클라이언트 초기화
openai_client = AsyncAzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version="2024-12-01-preview"
)

# 노트북 환경용 비동기 허용
nest_asyncio.apply()

async def llm_generate_text(prompt: str):
    response = await openai_client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_CHAT,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    return get_safe_content(response)

##### 3. 유틸: JSON 안전 파싱/재시도

In [0]:
# =========================
# 3) 유틸: JSON 안전 파싱/재시도
# =========================
def safe_json_loads(text: str) -> Optional[Dict[str, Any]]:
    try:
        return json.loads(text)
    except Exception:
        return None

def is_valid_playbook(obj: Dict[str, Any]) -> bool:
    keys = ["hook","angle","content_direction","cta","tone","urgency_level"]
    return all(k in obj and isinstance(obj[k], str) and obj[k].strip() for k in keys)

def extract_playbook_json(raw: str) -> Optional[str]:
    """
    모델이 JSON 외 텍스트를 섞어도, JSON 부분만 뽑아보는 보수적 처리
    """
    raw = raw.strip()
    # 가장 단순: 전체가 JSON이면 그대로
    obj = safe_json_loads(raw)
    if obj and is_valid_playbook(obj):
        return json.dumps(obj, ensure_ascii=False)

    # JSON 블록 추출 시도
    start = raw.find("{")
    end = raw.rfind("}")
    if start != -1 and end != -1 and end > start:
        chunk = raw[start:end+1]
        obj = safe_json_loads(chunk)
        if obj and is_valid_playbook(obj):
            return json.dumps(obj, ensure_ascii=False)

    return None

def parse_final_copy(raw: str) -> Tuple[Optional[str], Optional[str]]:
    """
    기대 포맷:
      - push_title: ...
      - push_body: ...
    """
    try:
        # LLM이 간혹 앞뒤에 ```json 같은 마크다운을 붙일 수 있으므로 정제
        json_str = raw.strip().replace("```json", "").replace("```", "").strip()
        data = json.loads(json_str)
        return data.get("push_title"), data.get("push_body"), data.get("email_title"), data.get("email_intro"), data.get("rec_list", [])
    except Exception as e:
        print(f"JSON 파싱 에러: {e}")
        return None, None, None, None, None

In [0]:
# llm response 방어 로직

def get_safe_content(response):
    # 1. response 자체가 유효한지 확인
    if not response or not response.choices:
        return "DEFAULT_MESSAGE: 응답 생성 실패"

    message = response.choices[0].message
    
    # 2. 최신 모델의 '거절' 필드 체크
    if hasattr(message, 'refusal') and message.refusal:
        return f"REFUSAL: {message.refusal}"
    
    # 3. content가 None이거나 빈 문자열인지 체크
    content = message.content
    if not content or content.strip() == "":
        return "DEFAULT_MESSAGE: 추천 콘텐츠를 확인해 보세요!" # 각 전략별 Fallback 메시지

    return content

##### 4. 콘텐츠 리스트 만들기(간단 추천)

In [0]:
# =========================
# 4) 콘텐츠 리스트 만들기(간단 추천)
#    - 최근 30일 top show_id 5개
# =========================
def build_content_list_for_users(latest_event_date: str, user_ids: List[int], k: int = 5) -> pd.DataFrame:
    """
    반환: pandas DF [user_id, content_list]
    content_list 예시:
      - 제목1 (키워드/장르)
      - 제목2 ...
    """
    # (1) 유저별 최근 30일 top show_id
    evt = (spark.table(EVT_TBL)
           .filter(F.col("event_date") <= F.lit(latest_event_date))
           .filter(F.col("event_date") > F.date_sub(F.lit(latest_event_date), 30))
           .filter(F.col("user_id").isin(user_ids))
           .groupBy("user_id", "show_id")
           .agg(F.sum("session_time").alias("watch_min"))
          )

    w = Window.partitionBy("user_id").orderBy(F.col("watch_min").desc())
    top = (evt.withColumn("rn", F.row_number().over(w))
              .filter(F.col("rn") <= k)
          )

    # (2) master에서 show_id -> title 가져오기 (컬럼명이 다를 수 있어 안전 처리)
    master = spark.table(CNT_TBL)

    # show_id 컬럼 후보들 중 존재하는 걸 선택(현장 파일마다 다름)
    master_cols = set(master.columns)
    show_id_col = "show_id" if "show_id" in master_cols else ("showId" if "showId" in master_cols else None)
    title_col_candidates = ["title", "name", "show_title", "primaryTitle"]
    title_col = next((c for c in title_col_candidates if c in master_cols), None)

    if show_id_col is None or title_col is None:
        # master 컬럼명이 예상과 다르면 show_id만으로 리스트 구성(최소 기능)
        joined = top.select("user_id", F.col("show_id").cast("string").alias("title"))
    else:
        joined = (top.join(master.select(F.col(show_id_col).alias("show_id_key"), F.col(title_col).alias("title")),
                           top.show_id == F.col("show_id_key"),
                           "left")
                    .select("user_id", F.coalesce(F.col("title"), F.col("show_id").cast("string")).alias("title"))
                 )

    # (3) user_id별 문자열 리스트 만들기
    content = (joined.groupBy("user_id")
                     .agg(F.collect_list("title").alias("titles"))
                     .withColumn(
                         "content_list",
                         F.expr("concat_ws('\n', transform(titles, x -> concat('- ', x)))")
                     )
                     .select("user_id", "content_list")
              )

    return content.toPandas()

##### 5. user_situation 만들기 (전략/행동 근거 포함)
모델 학습 단계에서 churn_reason 제외(2026-02-24)

In [0]:
# =========================
# 5) user_situation 만들기 (전략/행동 근거 포함)
# =========================
def build_user_situation(row: Dict[str, Any]) -> str:
    # snapshot 지표(없을 수 있어 fallback)
    dsll = row.get("days_since_last_login")
    wt7  = row.get("watch_time_7d_min")
    wt30 = row.get("watch_time_30d_min")
    seg  = row.get("segment")

    # campaign_targets
    strategy_code = row["strategy_code"]
    pb = row.get("probability_band")

    parts = [
        f"{strategy_code} 전략 대상 사용자입니다.",
        f"위험 단계(probability_band): {pb}"
    ]

    if seg is not None:
        parts.append(f"세그먼트: {seg}")
    if dsll is not None:
        parts.append(f"마지막 접속 이후 경과일: {dsll}일")
    if wt7 is not None and wt30 is not None:
        parts.append(f"최근 7일 시청시간: {wt7}분 / 최근 30일 시청시간: {wt30}분")

    return "\n".join(parts)

##### 6. 단건 처리: Playbook -> Final Copy

In [0]:
# =========================
# 6) 단건 처리: Playbook -> Final Copy
# =========================
async def process_one(row: Dict[str, Any], content_list: str) -> Dict[str, Any]:
    strategy_code = row["strategy_code"]
    if row.get("send_flag", 1) == 0 or strategy_code == "Calm":
        return {**row,
                "playbook_json": None,
                "push_title": None,
                "push_body": None,
                "email_title": None,
                "email_intro": None,
                "rec_list": None,
                "content_list": content_list,
                "skipped": True}

    user_situation = build_user_situation(row)

    # 1) Playbook JSON
    p_prompt = build_playbook_prompt(
        strategy_code=strategy_code,
        probability_band=row.get("probability_band", ""),
        priority_rank=int(row.get("priority_rank", 8))
    )

    playbook_json = None
    for _ in range(2):  # 2회 시도
        raw = await llm_generate_text(p_prompt)
        playbook_json = extract_playbook_json(raw)
        if playbook_json:
            break

    if not playbook_json:
        # 최소 안전 fallback (전략 가이드 기반)
        guide = STRATEGY_PLAYBOOK_GUIDE.get(strategy_code, {})
        fallback = {
            "hook": "오늘 딱 한 편만!",
            "angle": (guide.get("angles") or ["맞춤 추천"])[0],
            "content_direction": "최근 취향 기반 추천",
            "cta": (guide.get("cta_candidates") or ["지금 확인"])[0],
            "tone": (guide.get("tone_candidates") or ["friendly"])[0],
            "urgency_level": (guide.get("urgency_candidates") or ["medium"])[0],
        }
        playbook_json = json.dumps(fallback, ensure_ascii=False)

    # 2) Final Copy
    f_prompt = build_final_copy_prompt(
        user_situation=user_situation,
        strategy_code=strategy_code,
        playbook_json=playbook_json,
        content_list=content_list
    )

    raw2 = await llm_generate_text(f_prompt)
    title, body, email_title, email_intro, rec_list = parse_final_copy(raw2)

    # 마지막 안전장치
    if not title:
        title = "오늘의 추천이 도착했어요"
    if not body:
        body = "지금 바로 확인해보세요."
    if not email_title:
        email_title = "오늘의 추천이 도착했어요"
    if not email_intro:
        email_intro = "지금 바로 확인해보세요."
    if not rec_list:
        rec_list = []

    return {
        **row,
        "playbook_json": playbook_json,
        "push_title": title,
        "push_body": body,
        "email_title": email_title,
        "email_intro": email_intro,
        "rec_list": rec_list,
        "content_list": content_list,
        "skipped": False
    }

##### 7. 배치 실행: 최신 event_date 기준 send_flag=1만 처리

In [0]:
dbutils.widgets.text("latest_date", "")
param_date = dbutils.widgets.get("latest_date").strip()

# 원칙적으로 T-1(어제) 사용
t1 = spark.sql("SELECT date_sub(current_date(), 1) AS d").first()["d"]

snapshot_df = spark.table(TGT_TBL)

if param_date:
    latest_date = param_date
else:
    # T-1이 없을 수 있으니 "T-1 이하"에서 가장 최신 날짜로 fallback
    latest_date = (snapshot_df
                  .filter(F.col("event_date") <= F.lit(t1))
                  .agg(F.max("event_date").alias("max_d"))
                  .first()["max_d"])
    if latest_date is None:
        latest_date = t1

In [0]:
# =========================
# 7) 배치 실행: 최신 event_date 기준 send_flag=1만 처리
# =========================
async def run_generation(limit_users: int = 200, concurrency: int = 15):
    # 최신 event_date (targets 기준)
    latest_str = str(latest_date)

    # targets 로드
    tgt = (spark.table(TGT_TBL)
           .filter((F.col("event_date") == F.lit(latest_str)) & (F.col("probability_band") == F.lit("Critical")))
           .orderBy(F.col("priority_rank").asc())
           .limit(limit_users)
          )

    # 스냅샷 조인(문구 근거 강화)
    snap = (spark.table(SNP_TBL)
            .filter(F.col("event_date") == F.lit(latest_str))
            .select("event_date","user_id","segment","days_since_last_login","watch_time_7d_min","watch_time_30d_min")
           )

    df = (tgt.join(snap, ["event_date","user_id"], "left")
             .select("event_date","user_id","strategy_code","priority_rank","send_flag",
                     "probability_band",
                     "segment","days_since_last_login","watch_time_7d_min","watch_time_30d_min")
         )

    pdf = df.toPandas()
    if pdf.empty:
        print("No targets found.")
        return

    user_ids = pdf["user_id"].dropna().astype(int).tolist()
    content_df = build_content_list_for_users(latest_str, user_ids, k=5)
    content_map = dict(zip(content_df["user_id"], content_df["content_list"]))

    sem = asyncio.Semaphore(concurrency)

    async def _wrapped(row_dict):
        async with sem:
            uid = int(row_dict["user_id"])
            cl = content_map.get(uid, "- 추천 콘텐츠를 준비 중이에요")
            return await process_one(row_dict, cl)

    tasks = [_wrapped(r) for r in pdf.to_dict(orient="records")]
    results = await asyncio.gather(*tasks)

    out = pd.DataFrame(results)
    out["generated_ts"] = pd.Timestamp.utcnow()

    # 저장 (Delta)
    spark_out = spark.createDataFrame(out)
    (spark_out
        .select(
            F.col("event_date").cast("date").alias("event_date"),
            F.col("user_id").cast("bigint").alias("user_id"),
            "strategy_code",
            F.col("priority_rank").cast("int").alias("priority_rank"),
            "probability_band",
            "playbook_json",
            "push_title",
            "push_body",
            "email_title",
            "email_intro",
            "rec_list",
            "content_list",
            F.col("generated_ts").cast("timestamp").alias("generated_ts")
        )
        .write
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .option("mergeSchema", "true")
        .option("replaceWhere", f"event_date = '{latest_str}'")
        .format("delta")
        .saveAsTable(OUT_TBL)
    )

    print(f"Saved {len(out)} rows into {OUT_TBL} for event_date={latest_str}.")

##### 8. 실행
예시) 한 번에 200명까지 동시처리하는 함수 실행

In [0]:
# =========================
# 8) 실행
# =========================
await run_generation(limit_users=200, concurrency=15)

##### 메시지 이메일 전송

In [0]:
message_to_send_df = spark.table("signalcraft_databricks.default.dlt_silver_campaign_messages").where((F.col("event_date") == latest_date) & (F.col("strategy_code") != 'Calm'))

message_to_send_df.display()

In [0]:
USR_TBL = f"{CATALOG}.{SCHEMA}.dlt_bronze_user"
bronze_user_df = spark.table(USR_TBL)
message_to_send_df = spark.table("signalcraft_databricks.default.dlt_silver_campaign_messages").where((F.col("event_date") == latest_date) & (F.col("strategy_code") != 'Calm'))


result_df = (
    message_to_send_df.alias("m")
    .join(
        bronze_user_df.alias("u"), 
        message_to_send_df.user_id == bronze_user_df.user_id, 
        "leftouter"
    )
    .select("m.*", "u.age", "u.gender", "u.plan", "u.churn_date", "u.user_email")
)

result_df.display()

In [0]:
import requests, json, markdown

TMDB_API_KEY = dbutils.secrets.get(scope=SCOPE_NAME, key="tmdb-api-key").strip()

# 이미지 매칭 로직 (TMDB + 스마트 더미 Fallback)
def get_poster_url(title):
    try:
        # Movie 검색
        movie_url = f"https://api.themoviedb.org/3/search/movie?api_key={TMDB_API_KEY}&query={title}&language=ko-KR"
        res = requests.get(movie_url, timeout=5).json()
        if res.get('results') and res['results'][0].get('poster_path'):
            return f"https://image.tmdb.org/t/p/w500{res['results'][0]['poster_path']}"
        
        # TV/시리즈 검색
        tv_url = f"https://api.themoviedb.org/3/search/tv?api_key={TMDB_API_KEY}&query={title}&language=ko-KR"
        res_tv = requests.get(tv_url, timeout=5).json()
        if res_tv.get('results') and res_tv['results'][0].get('poster_path'):
            return f"https://image.tmdb.org/t/p/w500{res_tv['results'][0]['poster_path']}"
    except:
        pass
    
    # 실패 시 스마트 더미 이미지 (OTT 테마)
    return f"https://dummyimage.com/400x600/141414/E50914&text={title.replace(' ', '+')}"

# HTML 조립 로직
def build_html_body(email_title, email_intro, recommendations):
    # 상단부 및 스타일 (이전 템플릿 참조)
    html_start = f"""
    <body style="margin:0; background-color:#141414; font-family:'Noto Sans KR', sans-serif; color:#ffffff;">
        <center style="width:100%; background-color:#141414; padding: 40px 0;">
            <table width="600" style="margin:auto; background-color:#141414;">
                <tr><td style="padding:20px; text-align:center;"><h1 style="color:#E50914;">SignalCraft OTT</h1></td></tr>
                <tr>
                    <td style="padding:20px 30px;">
                        <h2 style="margin:0 0 10px;">{email_title}</h2>
                        <p style="font-size:16px; line-height:1.6; color:#cccccc; margin:0;">
                            {email_intro}
                        </p>
                    </td>
                </tr>
    """
    
    # 콘텐츠 리스트 (1 Featured + n Grid)
    content_html = ""
    for i, item in enumerate(recommendations[:len(recommendations)]):
        poster = get_poster_url(item['title'])
        if i == 0: # 첫 번째는 크게
            content_html += f"""
            <tr><td style="padding:10px; text-align:center;">
                <img src="{poster}" width="100%" style="border-radius:12px;">
                <h3>{item['title']}</h3><p style="color:#aaa;">{item['description']}</p>
            </td></tr><tr><td><table width="100%"><tr>
            """
        else: # 나머지는 2단 그리드
            content_html += f"""
            <td width="50%" style="padding:10px; vertical-align:top;">
                <img src="{poster}" width="100%" style="border-radius:8px;">
                <h4 style="margin:10px 0 5px;">{item['title']}</h4><p style="font-size:12px; color:#888;">{item['description']}</p>
            </td>
            """
            if i % 2 == 0 or i == len(recommendations[:len(recommendations)]) - 1:
                content_html += "</tr><tr>"
                
    html_end = """
            </tr></table></td></tr>
            <tr><td style="padding:40px; text-align:center;">
                <a href="https://www.netflix.com" style="background:#E50914; color:white; padding:15px 30px; text-decoration:none; border-radius:5px; font-weight:bold;">지금 시청하러 가기</a>
            </td></tr>
            </table>
        </center>
    </body>
    """
    return html_start + content_html + html_end

async def send_marketing_email(recipient_address, subject, email_intro, rec_list):
    # 1. 클라이언트 초기화
    connection_string = ACS_CONNECTION_STRING
    client = EmailClient.from_connection_string(connection_string)

    html_content = build_html_body(subject, email_intro, rec_list)

    try:
        # 2. 메시지 구성
        message = {
            "senderAddress": ACS_EMAIL_ADDRESS,
            "content": {
                "subject": subject,
                "html": html_content # AI가 생성한 마케팅 카피가 들어갈 자리
            },
            "recipients": {
                "to": [{"address": recipient_address}]
            }
        }

        # 3. 메일 발송 시작 (비동기)
        poller = await client.begin_send(message)
        result = await poller.result()
        
        # 객체(Attribute) 방식과 딕셔너리(Key) 방식을 모두 방어
        msg_id = getattr(result, 'message_id', None)  # 객체일 때
        if msg_id is None and isinstance(result, dict):
            msg_id = result.get('messageId') or result.get('id') # 딕셔너리일 때
            
        print(f"메일 발송 완료! Message ID: {msg_id}")
        return result

    except Exception as ex:
        print(f"메일 발송 중 오류 발생: {ex}")
    finally:
        # 4. 클라이언트 세션 닫기
        await client.close()

async def send_to_teams_webhook(subject, teams_body):
    webhook_url = TEAMS_WEBHOOK_URL.strip()

    payload = {
        "type": "message",
        "attachments": [
            {
                "contentType": "application/vnd.microsoft.card.adaptive",
                "content": {
                    "type": "AdaptiveCard",
                    "body": [
                        {
                            "type": "TextBlock",
                            "size": "Medium",
                            "weight": "Bolder",
                            "text": subject,
                            "style": "heading"
                        },
                        {
                            "type": "TextBlock",
                            "text": teams_body, # 생성한 변수 삽입
                            "wrap": True        # 자동 줄바꿈 활성화
                        }
                    ],
                    "actions": [
                        {
                            "type": "Action.OpenUrl",
                            "title": "지금 컨텐츠 보러 가기",
                            "url": "https://www.netflix.com/kr/"
                        }
                    ],
                    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
                    "version": "1.2"
                }
            }
        ]
    }

    # 4. HTTP POST 요청
    headers = {'Content-Type': 'application/json'}
    response = requests.post(webhook_url, data=json.dumps(payload), headers=headers)

    if response.status_code == 200 or response.status_code == 202:
        print("Adaptive Card 전송 완료!")
    else:
        print(f"오류 발생: {response.status_code}, {response.text}")

async def send_notifications(recipient_email, push_title, push_body, email_title, email_intro, rec_list):
    # 1. 이메일용: HTML로 변환 (markdown 모듈 사용)
    email_intro = markdown.markdown(email_intro)
    await send_marketing_email(recipient_email, email_title, email_intro, rec_list)
    
    # 2. Teams용: Markdown 그대로 사용 (\n만 체크)
    # Teams Webhook은 표준 Markdown을 잘 지원합니다.
    teams_body = push_body.replace('\n', '\n\n') # 가독성을 위해 줄바꿈 보강
    await send_to_teams_webhook(push_title, teams_body)

In [0]:
async def process_all_contexts(df):
    """
    데이터프레임의 각 행을 순회하며 질문을 던지는 메인 로직
    """
    # 
    tasks = []
    
    # 데이터프레임의 각 행을 순차적으로 읽음
    for index, row in df.iterrows():
        # 1. 행 데이터를 하나의 문자열(Context)로 변환
        user_email = row["user_email"] # 이메일
        push_title = row["push_title"] # 푸시 제목
        push_body = row["push_body"] # 푸시 본문
        email_title = row["email_title"] # 이메일 제목
        email_intro = row["email_intro"] # 이메일 본문
        rec_list = row["rec_list"] # 이메일 추천 컨텐츠 목록
        # 2. 비동기 작업 리스트에 추가
        # tasks.append(send_notifications(user_email, push_title, push_body, email_title, email_intro, rec_list)) #1건 테스트용 주석
        tasks.append(send_notifications("jwyoon082@gmail.com", push_title, push_body, email_title, email_intro, rec_list))
    
    # 3. 모든 작업을 병렬로 실행하고 결과를 수집
    # (순차적으로 던지고 싶다면 await를 루프 안에 넣으면 되지만, 성능상 gather를 추천합니다)
    results = await asyncio.gather(*tasks)
    
    return results

In [0]:
await process_all_contexts(result_df.limit(1).toPandas())